In [1]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import subprocess
import math
import time
from tqdm.auto import tqdm
from torchmetrics.functional import accuracy, f1_score, auroc
import random
from pathlib import Path
import os
import sys
from tqdm.auto import tqdm



In [2]:
sys.path.append('../training/')
import utils
from utils import precision_recall

In [4]:
cnn_list = []
kmerfre_list = []
ensembled_list = []
ensembled_possi = []
y_list = []
ID_list = []
for i in range(40):    
    test = utils.mydataset_ensembled("/fs/ess/PAS0439/MING/cilates_fungi_classifier/outputs/GutEuk_classifier/stage2/testing/", str(i).zfill(3), 1)   
    test_loader = DataLoader(dataset=test,
                    shuffle=False) 

    for _, (ID, cnn, kmerfre, y) in enumerate(test_loader): 

        ensembled = (cnn.view(-1) + kmerfre.view(-1))/2
        ensembled_pre = ensembled.argmax().item()
        cnn_pre = cnn.argmax().item()
        kmerfre_pre = kmerfre.argmax().item()
        ensembled_possibility = max(ensembled.numpy())
        
        cnn_list.append(cnn_pre)
        kmerfre_list.append(kmerfre_pre)
        ensembled_list.append(ensembled_pre)
        y_list.append(y.item())
        ID_list.append(ID[0][0])
        ensembled_possi.append(ensembled_possibility)

        
        


In [5]:
test_out = pd.DataFrame({"seq":ID_list , "cnn_out": cnn_list, "kmerfre_out": kmerfre_list, "ensembled_out": ensembled_list, "Y": y_list, "ensembled_possibility": ensembled_possi})

In [6]:
precision_recall(kmerfre_list, y_list)

Class 0: Precision: 99.34%| Recall: 95.47%
Class 1: Precision: 91.75%| Recall: 98.76%


In [7]:
precision_recall(cnn_list, y_list)

Class 0: Precision: 99.30%| Recall: 94.31%
Class 1: Precision: 89.84%| Recall: 98.70%


In [8]:
precision_recall(ensembled_list, y_list)

Class 0: Precision: 99.59%| Recall: 96.10%
Class 1: Precision: 92.85%| Recall: 99.22%


In [9]:
test_out.to_csv("/fs/ess/PAS0439/MING/cilates_fungi_classifier/test_res_stage2.csv", index = None)